In [23]:
import random

import numpy as np

import torch

In [24]:
# set seed for reproducibility

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.manual_seed(0)
random.seed(0)

In [25]:
x = torch.rand((4,4))
w1 = torch.rand((4,4))
w2 = torch.rand((4,4))
x, w1, w2

(tensor([[0.4963, 0.7682, 0.0885, 0.1320],
         [0.3074, 0.6341, 0.4901, 0.8964],
         [0.4556, 0.6323, 0.3489, 0.4017],
         [0.0223, 0.1689, 0.2939, 0.5185]]),
 tensor([[0.6977, 0.8000, 0.1610, 0.2823],
         [0.6816, 0.9152, 0.3971, 0.8742],
         [0.4194, 0.5529, 0.9527, 0.0362],
         [0.1852, 0.3734, 0.3051, 0.9320]]),
 tensor([[0.1759, 0.2698, 0.1507, 0.0317],
         [0.2081, 0.9298, 0.7231, 0.7423],
         [0.5263, 0.2437, 0.5846, 0.0332],
         [0.1387, 0.2422, 0.8155, 0.7932]]))

In [26]:
def get_original_result(input_data, weight1, weight2):
    a1 = torch.matmul(input_data, weight1)
    a2 = torch.matmul(a1, weight2)
    return a2

original = get_original_result(x, w1, w2)

In [27]:
def get_tensor_parallel_result(data_input, w1, w2):

    # column-wise split w1
    w1_1, w1_2 = w1.split(2, dim=1)

    # row-wise split w2
    w2_1, w2_2 = w2.split(2, dim=0)

    # upper side result
    # your code starts here
    upper_side_result = data_input.matmul(w1_1).matmul(w2_1)
    # your code ends here

    # lower side result
    # your code starts here
    lower_side_result = data_input.matmul(w1_2).matmul(w2_2)
    # your code ends here

    reduced_result = upper_side_result + lower_side_result

    return reduced_result

tp_result = get_tensor_parallel_result(x, w1, w2)

In [28]:
original, tp_result

(tensor([[0.8115, 1.7169, 2.0695, 1.6799],
         [1.2327, 2.2220, 3.0164, 2.3150],
         [0.9966, 1.9062, 2.4030, 1.8587],
         [0.5292, 0.8668, 1.2606, 0.9341]]),
 tensor([[0.8115, 1.7169, 2.0695, 1.6799],
         [1.2327, 2.2220, 3.0164, 2.3150],
         [0.9966, 1.9062, 2.4030, 1.8587],
         [0.5292, 0.8668, 1.2606, 0.9341]]))